# Mood Filtering Songs
In order to separate songs by mood, we would need to create categories of each moods and write functions to filter the dataframe for each category. For faster runtime, we will be creating a `sqlite3` database and calling on the filters using SQL commands.  

### Mood Categories
We distinguished our moods based on Robert Thayer's [traditional model of mood](https://sites.tufts.edu/eeseniordesignhandbook/2015/music-mood-classification/), which categorizes moods based on their valence (positive to negative valence) and energy (low to high energy). 

Because of this, we chose to filter songs based on the `energy` and `valence` features from Spotify API's audio features. According to the Spotify API, `energy` is the perceptual measure of intensity and activity, and `valence` is the musical positiveness in a track, both measured from 0.0 to 1.0.

## Creating SQL Database
To create a SQL, we first have to make sure we create a dataframe of related songs from the Spotify API. 

In [1]:
# import for spotify api
import pandas as pd
import spotipy
import re
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import time

# import for database
import sqlite3

In [2]:
results=pd.read_csv("results-1.csv") #gets results from csv file
related_words=["bakery"]+list(results["topic"]) +list(results["related_word"]) #adds all entries to one list
related_words=list(set(related_words)) #makes sure there no overlap

In [3]:
# ids needed to access spotify api
cid="45d772a85b0a4c2681a42696ad3b5ef3"
cis="9471f4a95ba34fb1a7c8414bc801542e"

In [4]:
sp_oauth=SpotifyClientCredentials(client_id=cid,client_secret=cis)

#this line doesn't work in deepnote for some reason but it works in jupyter lab
sp_oauth=SpotifyOAuth(client_id=cid,client_secret=cis,redirect_uri="http://localhost:2023/callback",scope=["playlist-modify-public","playlist-modify-private"])

sp = spotipy.Spotify(auth_manager=sp_oauth)

In [5]:
#add rows of the columns associated with the song features
df = pd.DataFrame(columns=["title","artist","release_date","uri","danceability", "energy", "valence"]) 

In [6]:
for word in related_words:
    word_results=sp.search(q=f"track:{word}", limit=10)

    for song in word_results["tracks"]["items"]:

        mod_title=" "+re.sub(r'[^\w\s]', '', song["name"].lower())+" "
        #checks that the actual word is in title, not just the word plus some letters
        if f" {word} " in mod_title:
            track_features = sp.audio_features(song["id"])
            try:
                try:
                    track_features[0]['acousticness'] # check if audio features exist
                except:
                    continue

                row=[
                    song["name"],
                    song["artists"][0]["name"],
                    song["album"]["release_date"],
                    song["uri"],
                    track_features[0]['danceability'],
                    track_features[0]['energy'],
                    track_features[0]['valence']
                ]

                # for this specific track id call that audio features thing and get those special features
                df.loc[len(df)]=row
            except spotipy.SpotifyException as e: # handle spotify api errors
                if (e.http_status == 429) | (e.code == -1):  # Too many requests
                    print("Error 429 - Too many requests")
                    print("Reason:", e)
                    print("Playlist cannot load at this moment. Please try again later.")
                    break
                else:
                    print("An error occurred:", e)
            except Exception as e: # handle general errors
                print("An error occurred:", e)
                break

In [7]:
# check length of dataframe
print(len(df))

508


In [8]:
# check database
df.head()

,title,artist,release_date,uri,danceability,energy,valence
0,COOK UP,Cochise,2022-12-09,spotify:track:2yUAogxEgxEHBGG3zw57x9,0.881,0.609,0.603
1,Apple Baking,CoComelon,2023-09-01,spotify:track:4qT2N5I4ET3VG31LFcUsNr,0.760,0.421,0.972
2,Baking the Wondertart,Kristofer Maddigan,2022-06-30,spotify:track:0Bfa2cjv16VkogySuQMIkd,0.235,0.659,0.263
3,The Baking Song,Alice's Wonderland Bakery - Cast,2022-02-11,spotify:track:7DTcRS9BeM3YERBVARX8xk,0.860,0.719,0.978
4,What Baking Can Do,Jessie Mueller,2016-06-03,spotify:track:0h77yfaXxrLMLz31uNDigN,0.704,0.553,0.415


## Creating SQL Database
We opted to use an SQL database in for faster runtimes in case there are big dataframes generated based on related words. To create an SQL database from the dataframe, we simply used the `to_sql()` function then create the database.

In [9]:
# create database connection
conn = sqlite3.connect("related_songs.db")

In [10]:
# create table in database
df.to_sql("songs", conn, if_exists="replace")

508

In [11]:
# check work
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall()) # to check that it is indeed in database

[('songs',)]


In [12]:
# close connection
conn.close()

## Filter songs 
To filter songs, we will be using customs functions written in `playlist_filter.py` that filter songs based on the aformentioned parameters.

In [13]:
# import self-made class
import playlist_filter as pf

# SQL database
conn = sqlite3.connect("related_songs.db")

In [14]:
hype = pf.query_find_hype(df, conn)
print("There are " + str(len(hype)) + " songs in this playlist.")
hype.head()

There are 181 songs in this playlist.


,title,artist,release_date,uri,danceability,energy,valence
0,COOK UP,Cochise,2022-12-09,spotify:track:2yUAogxEgxEHBGG3zw57x9,0.881,0.609,0.603
1,The Baking Song,Alice's Wonderland Bakery - Cast,2022-02-11,spotify:track:7DTcRS9BeM3YERBVARX8xk,0.860,0.719,0.978
2,Baking Shit,Bando KD,2023-04-21,spotify:track:2fddu5OI8axXBokQmMu6oW,0.662,0.583,0.834
3,Baking Soda,Tone!!!,2017-01-25,spotify:track:7JcVpOIUXcbTQH0PNH7dsa,0.751,0.712,0.511
4,Dessert,Dawin,2015-09-25,spotify:track:0mxoXsr14ILAYcXZBMJcOs,0.866,0.731,0.765


In [15]:
agitated = pf.query_find_agitated(df, conn)
print("There are " + str(len(agitated)) + " songs in this playlist.")
agitated.head()

There are 113 songs in this playlist.


,title,artist,release_date,uri,danceability,energy,valence
0,Baking the Wondertart,Kristofer Maddigan,2022-06-30,spotify:track:0Bfa2cjv16VkogySuQMIkd,0.235,0.659,0.263
1,What Baking Can Do,Jessie Mueller,2016-06-03,spotify:track:0h77yfaXxrLMLz31uNDigN,0.704,0.553,0.415
2,Dessert,Jim Gaffigan,2006-02-07,spotify:track:5QYArESi992DAsi97TQtZh,0.591,0.625,0.289
3,Bistro,Teide Garcia,2023-03-10,spotify:track:75BKUJhsdVhntSUFH3SJIA,0.642,0.516,0.109
4,Warehouse,Oliver Smith,2020-01-08,spotify:track:2wER3gVMMIWPusxOjoWlcQ,0.554,0.800,0.244


In [16]:
sorrowful = pf.query_find_sorrowful(df, conn)
print("There are " + str(len(sorrowful)) + " song in this playlist.")
sorrowful.head()

There are 128 song in this playlist.


,title,artist,release_date,uri,danceability,energy,valence
0,Get Baking,Tom Howe,2012-11-01,spotify:track:5igN5TAubJCQhpcqdHEhIJ,0.856,0.260,0.381
1,Baking Goodies in the Mad World,Brett Jamez,2023-11-15,spotify:track:71HZCPygm9vsYzMfY1agfK,0.708,0.479,0.225
2,Baking in the Summer,Penny and Sparrow,2022-01-21,spotify:track:0h7jJumeYrVTxO46xKudpk,0.268,0.290,0.214
3,What's For Dessert?,Nitta' Gritta',2022-05-27,spotify:track:3yGIwZB68iW9gjK0ggNG7S,0.623,0.319,0.274
4,Bistro,Lofi Fruits Music,2022-07-29,spotify:track:4cebuB8wGgXL62fnXYCyoz,0.667,0.184,0.116


In [17]:
chill = pf.query_find_chill(df, conn)
print("There are " + str(len(chill)) + " song in this playlist.")
chill.head()

There are 86 song in this playlist.


,title,artist,release_date,uri,danceability,energy,valence
0,Apple Baking,CoComelon,2023-09-01,spotify:track:4qT2N5I4ET3VG31LFcUsNr,0.760,0.421,0.972
1,Baking Soda,MAVI,2022-10-14,spotify:track:5cNx7JPZrX6bBUnrn1EmIr,0.786,0.499,0.690
2,Room For Dessert?,Lunchtime Jazz,2022-10-09,spotify:track:3H9G8UFwVWjYlac1zCgMzT,0.711,0.297,0.684
3,Bistro Fada,Stephane Wrembel,2011-05-13,spotify:track:08T5wMbEnPN8ghT8o24KY6,0.761,0.371,0.864
4,Bistro Fada,Stephane Wrembel,2011-12-09,spotify:track:1PnOzUB8CP0SVwVmMkqY3k,0.578,0.322,0.826


In [18]:
# close connection
conn.close()